In [2]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

In [3]:
# подгрузим один из методов этой библиотеки
from fake_useragent import UserAgent

In [107]:
def get_hrefs_from_page(page_number):
  page_url = f'https://shazoo.ru/tags/590/anime?page={page_number}'
  response = requests.get(page_url, headers={'User-Agent': UserAgent().chrome})

  if not response.ok:
    return []

  html = response.content
  soup = BeautifulSoup(html, 'html.parser')
  obj = soup.findAll('h4', attrs={'class': 'leading-normal'})

  hrefs_list = []

  for i, el in enumerate(obj):
    hrefs_list.append(el.a['href'])

  return hrefs_list

In [132]:
def get_all_data_from_post(post_url):
  response = requests.get(post_url, headers={'User-Agent': UserAgent().chrome})

  html = response.content
  soup = BeautifulSoup(html, 'html.parser')

  title_obj = soup.find('h1', attrs={'class': 'sm:max-w-4xl text-xl sm:text-3xl leading-tight font-bold break-words dark:text-gray-300'})
  title = title_obj.text


  tags_list = []
  tags_obj = soup.findAll('ul', attrs={'class': 'flex flex-wrap gap-x-2 gap-y-1'})
  tags_li_soup = BeautifulSoup(str(tags_obj), 'html.parser')
  tags_li_list = tags_li_soup.findAll('li' )

  for el in tags_li_list:
    tags_list.append(el.a.text)

  time_name_obj = soup.find('div', attrs={'class': 'flex items-center gap-2'})
  time = time_name_obj.time.text
  author = BeautifulSoup(str(time_name_obj), 'html.parser').findAll('a')[1].text

  return {'title': title, 'tags': tags_list, 'time': time, 'author': author}


In [147]:
def parse_data(n_pages=1, start_page=0):
  df = pd.DataFrame(columns=["title", "tags", "time", "author"])

  for i in range(start_page, n_pages):
    hrefs_list = get_hrefs_from_page(i)

    for post in hrefs_list:
      post_data = get_all_data_from_post(post)
      df = pd.concat([df, pd.DataFrame([post_data])], ignore_index=True)

      time.sleep(1)

    time.sleep(1)

  return df

In [150]:
df = parse_data(n_pages=10)

df.to_csv("ten_pages_result.csv")